In [ ]:
import re
import requests
#Get all download links for pgn files of games from the main page of The Week In Chess
url = "https://theweekinchess.com/twic"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}
response = requests.get(url, headers=headers)
html = response.text
pattern = r'<a href="(.*?)">PGN</a>'
matches = re.findall(pattern, html)

links = []
for match in matches:
    links.append(match)

links

In [ ]:
#Download pgn files, each contains around a thousand games
for link in links:
    !wget -P games $link

In [1]:
import re
import requests

#Download games from different tournaments that are recorded on fide's official websites

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}
links = []
for month in range(1, 13): #Go through all months
    for year in range(2015, 2023):
        for country in ['USA', 'RUS']:
            url = f'https://ratings.fide.com/tournament_list.phtml?moder=ev_code&country={country}&rating_period={year}-{month}-01'
            response = requests.get(url, headers=headers)
            html = response.text    
            pattern = 'event=(\d+)><img src=/imga/pgn.gif'
            matches = re.findall(pattern, html)    
            for match in matches:                
                links.append('https://ratings.fide.com/view_pgn.phtml?code=' + str(match) + '&download=1')
    
len(links)


KeyboardInterrupt



In [ ]:
for i, link in enumerate(links):
    response = requests.get(link, headers=headers)    
    file_name = 'unzipped/unzipped/' + str(i) + ".pgn"
    with open(file_name, 'w') as f:
        f.write(response.text)

In [ ]:
#Extract pgn files from zip
!unzip 'games/*.zip' -d unzipped

In [ ]:
#Zip into one folder that can be used next time instead of downloading from The Week In Chess again
!zip -r pgns.zip unzipped

In [ ]:
#Download player information from fide's website
!wget http://ratings.fide.com/download/players_list_xml.zip

In [ ]:
!unzip players_list_xml.zip

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET

# Parse the XML file
tree = ET.parse('players_list_xml_foa.xml')
root = tree.getroot()

In [ ]:
# Extract the data from XML and create a dictionary
#Turn the dictionary into a dataframe that contains player information
data = []
for child in root:
    player = {}
    for subchild in child:
        #print(subchild)
        player[subchild.tag] = subchild.text
    data.append(player)
df = pd.DataFrame(data)

In [ ]:
#Save dataframe for future use
df.to_csv('players_data.csv')